In [2]:
# Anomaly Graphing Functions
import numpy as np
import matplotlib as plt
import pandas as pd

#decYears and months lists required. This can be accessed from the dataset. Code to do it is below:
# Change the file path below to access any of the sea ice average text files
seaIceAvg1 = pd.read_csv("./data/seaIceAvg/seaIceAvg1.txt", sep="\t")
years = seaIceAvg1["#year"]
months = seaIceAvg1["month"]
decYears = seaIceAvg1["decimal_year"]

# All anomaly graph functions have three inputs: data, name, location. The data parameter is whatever data you want to make an anomaly plot of for monAnomalyGraph and seaAnomalyGraph. For runningMeanAnomaly, the data parameter is just the output of the monAnomalyGrpah function. So you can feed data into monAnomalyGraph, and then filter it as a running mean by putting that output as the input into runningMeanAnomaly
# The parameters for the last two functions are the two datasets you want to find the r^2 or slope for, and the y-coord is the y-coordinate of the location on the graph that you want to text to be displayed.

# This function creates an anomaly time series that takes each month (January 2023, for example) and finds the difference between it's sea ice area average and all of that month's sea ice area average (the average of all Januaries). The point is to find the difference between the average January and plot how different, or 'anomalous', it is.
def monAnomalyGraph(data, name, location):
    graphData = []
    for index, i in enumerate(decYears):
        if np.isnan(data[index]):
            graphData.append(0)
        else:
            monthTotal = []
            for jdex, j in enumerate(months):
                if j == months[index] and not np.isnan(data[jdex]):
                        monthTotal.append(data[jdex])
            graphData.append((data[index]-sum(monthTotal)/len(monthTotal)))

    fig = plt.figure(num=None, figsize=(10, 6))
    ax1 = plt.subplot(111)
    ax1.set_title("Monthly Anomaly Map for the " + name + " in the " + location)
    ax1.set_xlabel("Year")
    ax1.set_ylabel(name + " (Pa)")
    
    ax1.plot(decYears, graphData, linewidth=1.0, color='r')
    ax1.plot(decYears, [0] * len(decYears), linewidth=1.0, color='b')
    return graphData

# This function does the same thing as the monAnomalyGraph, except instead of finding the difference between a month and all of that month, it finds the difference betwen a season and the average of all of that season. This smoothens the data due to less data points. 
def seaAnomalyGraph(data, name, location):
    # Dec jan feb, march april may, june july august, september october november
    monthList = [[12, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]
    dataCopy = data#.fillna(0)
    graphData = []
    for index, i in enumerate(decYears):
        if np.isnan(data[index]):
            graphData.append(0)
        else:
            monthTotal = []
            season = 0
            for kdex, k in enumerate(monthList):
                if months[index] in k:
                    season = kdex
                    for jdex, j in enumerate(months):
                            if j in k and not np.isnan(dataCopy[jdex]):
                                monthTotal.append(dataCopy[jdex])
            seasonVal = 0
            for kdex, k in enumerate(years):
                if k == math.floor(i) and months[kdex] in monthList[season]:      
                    seasonVal += dataCopy[kdex]
            graphData.append(((seasonVal/3)-sum(monthTotal)/len(monthTotal)))

    fig = plt.figure(num=None, figsize=(10, 6))
    ax1 = plt.subplot(111)
    ax1.set_title("Seasonal Anomaly Map for the " + name + " in the " + location)
    ax1.set_xlabel("Year")
    ax1.set_ylabel(name + " (Pa)")

    ax1.plot(decYears, graphData, linewidth=1.0, color='r')
    ax1.plot(decYears, [0] * len(decYears), linewidth=1.0, color='b')
    return graphData

# This function takes the monAnomalyGraph output as an input, and creates a 12-month running mean. This takes each 12 points of that graph and combines it into 1 average. The first data point will be the average anomaly of January of one year to the January of the next, and the second data point will be the same but from February to the next February. This smoothens the data even further.
def runningMeanAnomaly(data, name, location):
    rmData = []
    for i in range(len(data)):
        yearTotal = []
        if i > (len(data)-12):
            for j in range(len(data)-12, len(data)):
                yearTotal.append(data[j])
        else:
            for j in range(0, 12):
                yearTotal.append(data[i+j])
        rmData.append(sum(yearTotal)/len(yearTotal))

    fig = plt.figure(num=None, figsize=(10, 6))
    ax1 = plt.subplot(111)
    ax1.set_title("Running Mean Anomaly Map for the " + name + " in the " + location)
    ax1.set_xlabel("Year")
    ax1.set_ylabel(name + " (Pa)")
    ax1.plot(decYears, rmData, linewidth=1.0, color='r')    
    ax1.plot(decYears, [0] * len(decYears), linewidth=1.0, color='b')
    return rmData

# This function does the same as the runningMeanAnomaly, however instead of taking the next 12 months, this takes the 6 months on either side. For any month, it'll take the average of it, the six months before it, and the six months after and plot it as a datapoint.
def sixMonthRunningMeanAnomaly(data, name, location):
    rmData = []
    for i in range(len(data)):
        yearTotal = 0
        if i > (len(data)-7):
            for j in range(len(data)-13, len(data)):
                yearTotal += data[j]
        elif i < 7:
            for j in range(0, 13):
                yearTotal += data[j]
        else:
            for j in range(i-6, i+7):
                yearTotal += data[j]
        rmData.append(yearTotal)

    fig = plt.figure(num=None, figsize=(10, 6))
    ax1 = plt.subplot(111)
    ax1.set_title("Running Mean Anomaly Map for the " + name + " in the " + location)
    
    ax1.plot(decYears, rmData, linewidth=1.0, color='r')    
    ax1.plot(decYears, [0] * len(decYears), linewidth=1.0, color='b')
    return rmData

# This function simply finds the r^2 score of two datasets and displays it on the graph.
def siaR2(data1, data2, yCoord):
    ax1.annotate("r^2 score: {:.5f}".format(pd.Series(data1).corr(pd.Series(data2))**2), (0, yCoord))

# This function finds the slope between two datasets and displays the slope on the graph.
def siaSlope(data1, data2, yCoord):
    m, b = np.polyfit(data1, data2, deg=1)
    ax1.plot(np.array(data1), m*np.array(data1)+b, color="red")
    ax1.annotate("slope: {:.15f}".format(m), (0, yCoord))

FileNotFoundError: [Errno 2] No such file or directory: './data/seaIceAvg/seaIceAvg1.txt'